<a href="https://colab.research.google.com/gist/mariateodorapopescu/b1d12f21b5d7c51bdb1a48a101ad18b7/entity-resolution-veridion-challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entity Resolution challenge

By Popescu Maria-Teodora

E: popescumariateodora@yahoo.com

T: +40787763178

Github: https://github.com/mariateodorapopescu

LinkedIn: www.linkedin.com/in/mariateodorapopescu

## Setup

### Getting the data

I saved the dataset to personal folder, even though I just could take it from your drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

### Installing dependencies

Comment the instructions below if you already have them installed

In [2]:
! pip install pandas pyarrow
! pip install fuzzywuzzy python-Levenshtein

### Imports

In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import json
from fuzzywuzzy import fuzz, process
import re
from collections import defaultdict
import copy

## Functions

In [4]:
def process_value(value):
    '''
    Processes a value that may contain multiple entries separated by "|".

    Parameters:
        value (str or any): The input value. If it's a string that contains '|', it will be split into a list.
                            Otherwise, it returns the value unchanged.

    Returns:
        list or original value: A list of cleaned strings if split is needed, or the original value otherwise.
    '''
    # If the value is a string and contains '|', it's treated as a list of values
    if isinstance(value, str) and "|" in value:
        # Split by "|" and remove surrounding spaces from each item
        return [item.strip() for item in value.split("|")]
    return value


def row_to_dict(row):
    '''
    Converts a Pandas DataFrame row into a dictionary, while handling multi-value fields.

    Parameters:
        row (pd.Series): A row from a DataFrame.

    Returns:
        dict: A dictionary with each value processed through `process_value`.
    '''
    result = {}
    # For each column in the row, apply the processing function
    for col, value in row.items():
        result[col] = process_value(value)
    return result


def clean_text(text):
    '''
    Cleans a string or a list of strings by:
    - Converting to lowercase
    - Stripping extra whitespace
    - Removing punctuation and special characters

    Parameters:
        text (str, list, or None): The input text or list of texts to clean.

    Returns:
        str or list: Cleaned version of the input.
    '''
    # If the value is missing or empty, return empty string
    if not text or text == "":
        return ""

    # If it's a list, clean each item recursively
    if isinstance(text, list):
        return [clean_text(item) for item in text if item]

    # Convert to string (in case it's not), lowercase, and strip spaces
    text = str(text).lower().strip()

    # Remove all non-alphanumeric characters except whitespace
    text = re.sub(r'[^\w\s]', '', text)

    return text


def get_similarity_score(row1, row2):
    '''
    Calculates a similarity score between two DataFrame rows based on:
    - Fuzzy match of company names
    - Bonus if country and city are identical

    Parameters:
        row1 (pd.Series): The first company record.
        row2 (pd.Series): The second company record.

    Returns:
        int: A similarity score (max 120), composed of name similarity + location bonus.
    '''
    # Use fuzzy string matching for name similarity
    name_sim = fuzz.token_sort_ratio(row1['company_name_clean'], row2['company_name_clean'])

    # Check if both country and city match exactly
    location_match = (
        row1['main_country_clean'] == row2['main_country_clean'] and
        row1['main_city_clean'] == row2['main_city_clean'] and
        row1['main_country_clean'] != ''
    )

    # Give bonus points if location also matches
    bonus = 20 if location_match else 0

    return name_sim + bonus


def calculate_similarity(comp1, comp2):
    '''
    Calculates a similarity score between two company dictionaries.
    Similar to `get_similarity_score`, but works with dicts instead of Pandas rows.

    Parameters:
        comp1 (dict): First company record.
        comp2 (dict): Second company record.

    Returns:
        int: Total similarity score.
    '''
    # Compare cleaned names using fuzzy matching
    name_sim = fuzz.token_sort_ratio(comp1['name_clean'], comp2['name_clean'])

    # Assume no location match initially
    location_match = False

    # Check if country and city are exactly the same
    if comp1['country_clean'] and comp2['country_clean'] and comp1['country_clean'] == comp2['country_clean']:
        if comp1['city_clean'] and comp2['city_clean'] == comp2['city_clean']:
            location_match = True

    # Add bonus if location is an exact match
    bonus = 20 if location_match else 0

    return name_sim + bonus


def select_best_record(group):
    '''
    Selects the most complete record (row) from a group of company entries.
    Completeness is measured by counting non-null fields.

    Parameters:
        group (pd.DataFrame): A group of rows from the DataFrame.

    Returns:
        pd.Series: The row with the most non-null values.
    '''
    # Create a temporary column with a score based on how many non-null fields each row has
    group['completeness'] = group.notna().sum(axis=1)

    # Return the row with the highest completeness score
    return group.loc[group['completeness'].idxmax()]


def completeness_score(company):
    '''
    Calculates a "completeness score" for a single company dictionary.
    Higher scores indicate more filled-in data.

    Parameters:
        company (dict): A dictionary containing a company's data.

    Returns:
        float: The completeness score for the company.
    '''
    score = 0

    # Step 1: Core identity fields (name, location) → 2 points each
    basic_fields = ['company_name', 'company_legal_names', 'company_commercial_names',
                    'main_country', 'main_city', 'main_region']
    for field in basic_fields:
        if field in company and company[field] and company[field] != "":
            score += 2

    # Step 2: Contact fields → 3 points each
    contact_fields = ['phone_numbers', 'primary_email', 'website_url', 'website_domain']
    for field in contact_fields:
        if field in company and company[field] and company[field] != "":
            score += 3

    # Step 3: Descriptions → 2 points each
    desc_fields = ['short_description', 'long_description']
    for field in desc_fields:
        if field in company and company[field] and company[field] != "":
            score += 2

    # Step 4: Financial information → 3 points each if non-zero
    if 'revenue' in company and company['revenue'] != 0:
        score += 3
    if 'employee_count' in company and company['employee_count'] != 0:
        score += 3

    # Step 5: Bonus for any other non-empty/non-zero field → 0.5 points each
    excluded_fields = basic_fields + contact_fields + desc_fields + ['revenue', 'employee_count']
    for field, value in company.items():
        if field not in excluded_fields and value not in ("", 0, None, []):
            score += 0.5

    return score

## Verifying the reading of the input

Reading/getting and storing the data

In [5]:
# getting and storing the data
df = pd.read_parquet('/content/drive/MyDrive/ceva/veridion_entity_resolution_challenge.snappy.parquet')

Let's see now some statistics about the rows

In [6]:
df.describe()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
count,32617,6890,28121,31415,31415,30112,5979,29602,23820,19979,...,19468,19445,33446,11592,11592,7229,7229,8723,8723,6151
unique,18715,4406,16840,137,138,953,3018,6673,10284,10304,...,18580,18082,1,7096,7096,5452,2,223,2,262
top,Fresh Burger,China Merchants Securities Co. Ltd.,Fresh Burger,US,United States,England,Manhattan,London,2000,Main Street,...,Advivision is a telecommunications consulting ...,Custom Programming Services | Software Develop...,Active,indianspices.com,indianspices.com,400298,modelled,15,modelled,1
freq,30,22,30,10119,10119,2599,105,357,44,140,...,16,22,33446,7,7,23,6147,1370,4615,316


Seeing other info out of it, such as data types

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33446 entries, 0 to 33445
Data columns (total 75 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   company_name                  32617 non-null  object
 1   company_legal_names           6890 non-null   object
 2   company_commercial_names      28121 non-null  object
 3   main_country_code             31415 non-null  object
 4   main_country                  31415 non-null  object
 5   main_region                   30112 non-null  object
 6   main_city_district            5979 non-null   object
 7   main_city                     29602 non-null  object
 8   main_postcode                 23820 non-null  object
 9   main_street                   19979 non-null  object
 10  main_street_number            17034 non-null  object
 11  main_latitude                 17031 non-null  object
 12  main_longitude                17031 non-null  object
 13  main_address_raw

In [8]:
print(df.dtypes)

company_name                object
company_legal_names         object
company_commercial_names    object
main_country_code           object
main_country                object
                             ...  
revenue                     object
revenue_type                object
employee_count              object
employee_count_type         object
inbound_links_count         object
Length: 75, dtype: object


Seeing the first 5 records of it

In [9]:
df.head()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Owens Liquors is a retail establishment locate...,Retail Trade | Liquor Stores | Wine & Liquor,Active,None,None,None,None,None,None,None
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,None,None,Active,None,None,None,None,9,extracted,None
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,AAA Auto Otrokovice (Zlín) is a car dealership...,In-store Shopping | Investment Management Serv...,Active,None,None,None,None,None,None,None
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,None,None,Active,None,None,None,None,None,None,None
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,None,None,Active,None,None,None,None,None,None,None


## Cleaning the data

In [10]:
# ----------------------------------------
# 1. Identify numeric and text columns
# ----------------------------------------

# These are columns we *know* are numeric (based on schema/domain knowledge)
known_numeric_columns = [
    'main_street_number', 'inbound_links_count',
    'main_latitude', 'main_longitude', 'num_locations',
    'year_founded', 'lnk_year_founded',
    'naics_2022_primary_code', 'naics_2022_secondary_codes',
    'revenue', 'employee_count'
]

# Attempt to auto-detect additional numeric columns
potential_numeric_columns = []

for col in df.columns:
    # Focus only on object-type columns that aren't already known as numeric
    if df[col].dtype == 'object' and col not in known_numeric_columns:
        # Sample first 100 non-null values
        sample = df[col].dropna().head(100)
        if len(sample) > 0:
            try:
                # Try converting sample to numeric
                pd.to_numeric(sample)
                # If successful, mark column as potentially numeric
                potential_numeric_columns.append(col)
            except:
                pass  # Ignore columns that fail to convert

# Combine known and potential numeric columns (removing duplicates)
all_numeric_columns = list(set(known_numeric_columns + potential_numeric_columns))

# All other columns are considered text
text_columns = [col for col in df.columns if col not in all_numeric_columns]

# ----------------------------------------
# 2. Fill missing values across related columns
# ----------------------------------------

# Fill `company_commercial_names` with `company_legal_names` when missing
mask = df['company_commercial_names'].isna() & df['company_legal_names'].notna()
df.loc[mask, 'company_commercial_names'] = df.loc[mask, 'company_legal_names']

# Fill `company_legal_names` with `company_commercial_names` when missing
mask = df['company_legal_names'].isna() & df['company_commercial_names'].notna()
df.loc[mask, 'company_legal_names'] = df.loc[mask, 'company_commercial_names']

# If both legal and commercial names are missing, fall back to `company_name`
mask = (
    df['company_legal_names'].isna() &
    df['company_commercial_names'].isna() &
    df['company_name'].notna()
)
df.loc[mask, 'company_legal_names'] = df.loc[mask, 'company_name']
df.loc[mask, 'company_commercial_names'] = df.loc[mask, 'company_name']

# Fill `main_city_district` with `main_city` if missing
mask = df['main_city_district'].isna() & df['main_city'].notna()
df.loc[mask, 'main_city_district'] = df.loc[mask, 'main_city']

# ----------------------------------------
# 3. Handle missing data
# ----------------------------------------

# Numeric columns: convert to numeric and replace NaN with 0
for col in all_numeric_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
        except Exception:
            pass  # Skip columns that cause issues during conversion

# Text columns: replace NaN with empty string
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna("")

# ----------------------------------------
# 4. (Optional) Validation - Check how many values were filled
# ----------------------------------------

# Count how many 0s exist in the first few numeric columns
for col in all_numeric_columns[:5]:
    if col in df.columns:
        zero_count = (df[col] == 0).sum()
        print(f"{col} → zero count: {zero_count}")

# Count how many empty strings exist in the first few text columns
for col in text_columns[:5]:
    if col in df.columns:
        empty_count = (df[col] == "").sum()
        print(f"{col} → empty string count: {empty_count}")

main_longitude → zero count: 16415
revenue → zero count: 26217
main_street_number → zero count: 18541
naics_2022_secondary_codes → zero count: 33202
website_number_of_pages → zero count: 27479
company_name → empty string count: 829
company_legal_names → empty string count: 829
company_commercial_names → empty string count: 829
main_country_code → empty string count: 2031
main_country → empty string count: 2031


And also veryfing after the cleaning

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33446 entries, 0 to 33445
Data columns (total 75 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   company_name                  33446 non-null  object 
 1   company_legal_names           33446 non-null  object 
 2   company_commercial_names      33446 non-null  object 
 3   main_country_code             33446 non-null  object 
 4   main_country                  33446 non-null  object 
 5   main_region                   33446 non-null  object 
 6   main_city_district            33446 non-null  object 
 7   main_city                     33446 non-null  object 
 8   main_postcode                 33446 non-null  object 
 9   main_street                   33446 non-null  object 
 10  main_street_number            33446 non-null  float64
 11  main_latitude                 33446 non-null  float64
 12  main_longitude                33446 non-null  float64
 13  m

In [12]:
df.describe()

,main_street_number,main_latitude,main_longitude,num_locations,year_founded,lnk_year_founded,naics_2022_primary_code,naics_2022_secondary_codes,primary_phone,website_number_of_pages,revenue,employee_count,inbound_links_count
count,3.344600e+04,33446.000000,33446.000000,33446.000000,33446.000000,33446.000000,33446.000000,33446.000000,3.344600e+04,33446.000000,3.344600e+04,33446.000000,33446.000000
mean,3.076009e+04,18.525348,-16.120452,0.901931,299.798003,128.082162,292391.268343,3817.743497,5.367803e+11,6.004006,6.078157e+06,22.393111,6.004006
std,5.454165e+06,23.813581,50.841302,18.142270,712.914270,489.702427,294034.431808,47442.769829,1.328591e+13,32.240867,2.924259e+08,638.411953,32.240867
min,0.000000e+00,-51.578219,-158.000329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000,-46.910894,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
50%,0.000000e+00,0.000000,0.000000,1.000000,0.000000,0.000000,238910.000000,0.000000,1.630584e+10,0.000000,0.000000e+00,0.000000,0.000000
75%,1.070000e+02,42.081438,0.000000,1.000000,0.000000,0.000000,541611.000000,0.000000,7.389244e+10,0.000000,0.000000e+00,1.000000,0.000000
max,9.974709e+08,65.351778,176.918711,3000.000000,2023.000000,2023.000000,925120.000000,813940.000000,9.718009e+14,1474.000000,3.677500e+10,66000.000000,1474.000000


In [13]:
df.head()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
0,Owens Liquors,Owens Liquors,Owens Liquors,US,United States,South Carolina,Pawleys Island,Pawleys Island,29585,Ocean Highway,...,Owens Liquors is a retail establishment locate...,Retail Trade | Liquor Stores | Wine & Liquor,Active,,,0.0,,0.0,,0.0
1,Club Tarneit,Club Tarneit,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,,...,,,Active,,,0.0,,9.0,extracted,0.0
2,AAA Auto Otrokovice Zlín,AAA Auto Otrokovice Zlín,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,AAA Auto Otrokovice (Zlín) is a car dealership...,In-store Shopping | Investment Management Serv...,Active,,,0.0,,0.0,,0.0
3,Gisinger GmbH,Gisinger GmbH,Gisinger GmbH,DE,Germany,Baden-Württemberg,Ühlingen-Birkendorf,Ühlingen-Birkendorf,79777,Berauer Straße,...,,,Active,,,0.0,,0.0,,0.0
4,Kasana Life,Kasana Life,Kasana Life,US,United States,Connecticut,Litchfield,Litchfield,06759,,...,,,Active,,,0.0,,0.0,,0.0


## Dictionary (JSON) approach

In [14]:
# ----------------------------------------
# 1. Convert DataFrame rows to dictionaries
# ----------------------------------------

# Convert each row into a dictionary using the `row_to_dict` function
# This will also handle splitting multi-value fields into lists
rows_as_dicts = []
for _, row in df.iterrows():
    row_dict = row_to_dict(row)
    rows_as_dicts.append(row_dict)

# ----------------------------------------
# 2. Save processed data to JSON
# ----------------------------------------

# Save the list of dictionaries to a JSON file for future use
# Useful for API input, interoperability, or storing structured data
with open('/content/drive/MyDrive/ceva/company_records.json', 'w', encoding='utf-8') as f:
    json.dump(rows_as_dicts, f, ensure_ascii=False, indent=2)

# ----------------------------------------
# 3. Preview a sample record
# ----------------------------------------

print("First row preview:")

# Select the first row as an example
example = rows_as_dicts[0]

# Print a few selected fields to manually verify the transformation
sample_fields = [
    'company_name',
    'main_country',
    'phone_numbers',
    'isic_v4_codes',
    'generated_business_tags',
    'domains'
]

for field in sample_fields:
    if field in example:
        print(f"{field}: {example[field]}")

# ----------------------------------------
# 4. Basic statistics on multi-value fields
# ----------------------------------------

print("\nSome statistics about the columns with multiple values:")

multiple_value_counts = {}

# Analyze the first 1000 records to count how many rows have lists for each field
for d in rows_as_dicts[:1000]:
    for key, value in d.items():
        if isinstance(value, list):
            if key not in multiple_value_counts:
                multiple_value_counts[key] = 0
            multiple_value_counts[key] += 1

# Sort fields by how often they contain multiple values
sorted_counts = sorted(multiple_value_counts.items(), key=lambda x: x[1], reverse=True)

# Display top 10 fields with the highest number of list-type values
for key, count in sorted_counts[:10]:
    print(f"{key}: {count} rows with multiple values")

First row preview:
company_name: Owens Liquors
main_country: United States
phone_numbers: +18433140354
isic_v4_codes: ['4722', '4781', '4791', '4799']
generated_business_tags: ['Retail Trade', 'Liquor Stores', 'Wine & Liquor']
domains: 

Some statistics about the columns with multiple values:
generated_business_tags: 560 rows with multiple values
nace_rev2_codes: 321 rows with multiple values
nace_rev2_labels: 321 rows with multiple values
sic_codes: 261 rows with multiple values
sic_labels: 261 rows with multiple values
business_tags: 222 rows with multiple values
isic_v4_codes: 162 rows with multiple values
isic_v4_labels: 162 rows with multiple values
phone_numbers: 95 rows with multiple values
company_commercial_names: 91 rows with multiple values


In [15]:
# ------------------------------------------------------------
# 1. Load the raw company data from a JSON file
# ------------------------------------------------------------

with open('/content/drive/MyDrive/ceva/company_records.json', 'r', encoding='utf-8') as f:
    companies = json.load(f)

print(f"Total companies loaded: {len(companies)}")

# ------------------------------------------------------------
# 2. Clean fields relevant for comparison
# ------------------------------------------------------------

# For each company record, we generate "clean" versions of fields
# These will help match companies based on text similarity
for company in companies:
    # Cleaned company name (lowercased, no punctuation)
    company['name_clean'] = clean_text(company['company_name'])

    # Cleaned domain if it exists
    if 'website_domain' in company and company['website_domain']:
        company['domain_clean'] = clean_text(company['website_domain'])
    else:
        company['domain_clean'] = ""

    # Cleaned country and city fields
    company['country_clean'] = clean_text(company['main_country'])
    company['city_clean'] = clean_text(company['main_city'])

# ------------------------------------------------------------
# 3. Group companies by domain name (most reliable method)
# ------------------------------------------------------------

domain_groups = defaultdict(list)   # domain → list of company indices
domain_assigned = set()            # Set of already grouped company indices

for idx, company in enumerate(companies):
    if company['domain_clean']:
        domain_key = company['domain_clean']
        domain_groups[domain_key].append(idx)
        domain_assigned.add(idx)

print(f"Domain-based groups found: {len(domain_groups)}")
print(f"Companies assigned via domain: {len(domain_assigned)}")

# ------------------------------------------------------------
# 4. Group remaining companies by name + location similarity
# ------------------------------------------------------------

name_groups = defaultdict(list)  # name_group_id → list of indices
name_assigned = set()            # Set of company indices matched by name
threshold = 80                   # Minimum similarity score to consider a match

# Blocking: group companies by country to reduce comparison complexity
country_blocks = defaultdict(list)
for idx, company in enumerate(companies):
    if idx not in domain_assigned and company['country_clean']:
        country_blocks[company['country_clean']].append(idx)

# Within each country block, compare companies pairwise
for country, indices in country_blocks.items():
    processed = set()

    for i, idx1 in enumerate(indices):
        if idx1 in processed:
            continue

        group = [idx1]
        processed.add(idx1)

        for idx2 in indices[i + 1:]:
            if idx2 not in processed:
                similarity = calculate_similarity(companies[idx1], companies[idx2])
                if similarity >= threshold:
                    group.append(idx2)
                    processed.add(idx2)

        # Only save group if more than 1 match found
        if len(group) > 1:
            group_id = f"name_group_{len(name_groups)}"
            name_groups[group_id] = group
            for idx in group:
                name_assigned.add(idx)

print(f"Name+location-based groups found: {len(name_groups)}")
print(f"Companies assigned via name/location: {len(name_assigned)}")

# ------------------------------------------------------------
# 5. Assign group IDs to each company
# ------------------------------------------------------------

for idx, company in enumerate(companies):
    if idx in domain_assigned:
        # Assign domain group ID
        for domain, group in domain_groups.items():
            if idx in group:
                company['company_group_id'] = f"domain_{domain}"
                break
    elif idx in name_assigned:
        # Assign name-location group ID
        for name_key, group in name_groups.items():
            if idx in group:
                company['company_group_id'] = name_key
                break
    else:
        # If the company wasn't grouped, assign a unique ID
        company['company_group_id'] = f"single_{idx}"

# ------------------------------------------------------------
# 6. Select the best (most complete) record from each group
# ------------------------------------------------------------

# Group company indices by group ID
grouped_companies = defaultdict(list)
for idx, company in enumerate(companies):
    grouped_companies[company['company_group_id']].append(idx)

best_records = []

for group_id, indices in grouped_companies.items():
    if len(indices) == 1:
        # If there's only one record in the group, it's automatically the best
        best_idx = indices[0]
    else:
        # Otherwise, calculate completeness score for each and choose the best
        scores = [completeness_score(companies[idx]) for idx in indices]
        best_idx = indices[scores.index(max(scores))]

    best_records.append(best_idx)
    companies[best_idx]['is_best_record'] = True  # Flag best record

# Mark all other records as not best
for idx in range(len(companies)):
    if idx not in best_records:
        companies[idx]['is_best_record'] = False

print(f"Total groups created: {len(grouped_companies)}")
print(f"Best records selected: {len(best_records)}")

# ------------------------------------------------------------
# 7. Save the full dataset (with group IDs and best flags)
# ------------------------------------------------------------

with open('/content/drive/MyDrive/ceva/entity_resolution_result.json', 'w', encoding='utf-8') as f:
    json.dump(companies, f, ensure_ascii=False, indent=2)

# ------------------------------------------------------------
# 8. Extract only the "best" records for each group
# ------------------------------------------------------------

best_companies = []

for company in companies:
    if company.get('is_best_record', False):
        # Create a copy to avoid modifying the original object
        best_company = company.copy()

        # Remove helper fields used only for internal logic
        keys_to_remove = ['is_best_record', 'name_clean', 'domain_clean', 'country_clean', 'city_clean']
        for key in keys_to_remove:
            best_company.pop(key, None)

        # Ensure certain fields are strings, not lists
        single_value_fields = [
            'company_name', 'main_country', 'main_city', 'main_region',
            'website_domain', 'website_url', 'primary_email'
        ]
        for field in single_value_fields:
            if isinstance(best_company.get(field), list):
                # Convert list to single space-separated string
                best_company[field] = " ".join(best_company[field])

        best_companies.append(best_company)

print(f"Unique companies after deduplication: {len(best_companies)}")

# ------------------------------------------------------------
# 9. Save the final set of unique companies (JSON + CSV)
# ------------------------------------------------------------

# Save to JSON
with open('/content/drive/MyDrive/ceva/unique_companies.json', 'w', encoding='utf-8') as f:
    json.dump(best_companies, f, ensure_ascii=False, indent=2)

# Convert to DataFrame for tabular use
unique_df = pd.DataFrame(best_companies)

# Save to CSV
unique_df.to_csv('/content/drive/MyDrive/ceva/unique_companies.csv', index=False)

# Preview
unique_df.head()

Total companies loaded: 33446
Domain-based groups found: 6579
Companies assigned via domain: 31893
Name+location-based groups found: 196
Companies assigned via name/location: 626
Total groups created: 7702
Best records selected: 7702
Unique companies after deduplication: 7702


,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count,company_group_id
0,Kasana Life,Kasana Life,Kasana Life,US,United States,Connecticut,Litchfield,Litchfield,06759,,...,,Active,,,0.0,,0.0,,0.0,name_group_6
1,CENTRO DE DIA QUIMEY CO Gral,"[CENTRO DE DIA QUIMEY CO Gral, Centro de Día Q...","[CENTRO DE DIA QUIMEY CO Gral, Centro de Día Q...",AR,Argentina,Buenos Aires,Ituzaingó,Ituzaingó,1713,Correa,...,"[Occupational Therapy, Therapy Services, Care ...",Active,centrosquimeyco.com.ar,centrosquimeyco.com.ar,680391.0,modelled,75.0,modelled,9.0,domain_centrosquimeycocomar
2,Dr Piave,Zafari Inc.,"[Dr Piave, Piave Pitisci Lake M.D., Dr Piave L...",US,United States,South Carolina,Mount Pleasant,Mount Pleasant,29464,Church Street,...,"[Psychiatry Services, Womens Reproductive Ment...",Active,drpiave.com,drpiave.com,143858.0,modelled,1.0,extracted,9.0,domain_drpiavecom
3,NOB,NOB NET ON BRAZIL LTDA,NOB,BR,Brazil,São Paulo,Jardim Helena,Taboão Da Serra,06765-000,Estrada São Francisco,...,"[Accounts Payable Management, App Consultation...",Active,nob.com.br,nob.com.br,3228518.0,modelled,75.0,modelled,4.0,domain_nobcombr
4,RSPCA Radcliffe-on-Trent Animal Centre,"[RSPCA Radcliffe-on-Trent Animal Centre, RSPCA...","[RSPCA Radcliffe-on-Trent Animal Centre, RSPCA...",GB,United Kingdom,England,Radcliffe-on-Trent,Nottingham,NG12 2DW,Nottingham Road,...,"[Adoption Rooms for Rescue Dogs, Legal Assista...",Active,rspca-radcliffe.org.uk,rspca-radcliffe.org.uk,0.0,,0.0,,41.0,domain_rspcaradcliffeorguk
